## A simple demostration of how to run

In [ ]:
# before import, make sure FaceBoxes and Sim3DR are built successfully, e.g.,
# sh build.sh

import cv2
import yaml

from FaceBoxes import FaceBoxes
from TDDFA import TDDFA
from utils.functions import draw_landmarks
from utils.render import render
from utils.depth import depth

import matplotlib.pyplot as plt

### Load configs

In [ ]:
# load config
cfg = yaml.load(open('configs/mb1_120x120.yml'), Loader=yaml.SafeLoader)

# Init FaceBoxes and TDDFA, recommend using onnx flag
onnx_flag = True  # or True to use ONNX to speed up
if onnx_flag:
    import os
    os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
    os.environ['OMP_NUM_THREADS'] = '4'
    
    from FaceBoxes.FaceBoxes_ONNX import FaceBoxes_ONNX
    from TDDFA_ONNX import TDDFA_ONNX
    
    face_boxes = FaceBoxes_ONNX()
    tddfa = TDDFA_ONNX(**cfg)
else:
    tddfa = TDDFA(gpu_mode=False, **cfg)
    face_boxes = FaceBoxes()

In [ ]:
# given an image path
img_fp = 'examples/inputs/emma.jpg'
img = cv2.imread(img_fp)
plt.imshow(img[..., ::-1])

### Detect faces using FaceBoxes

In [ ]:
# face detection
boxes = face_boxes(img)
print(f'Detect {len(boxes)} faces')
print(boxes)

### Regressing 3DMM parameters, reconstruction and visualization

In [ ]:
# regress 3DMM params
param_lst, roi_box_lst = tddfa(img, boxes)
# print(param_lst)
# print(roi_box_lst)

In [ ]:
import numpy as np
# reconstruct vertices and visualizing sparse landmarks
dense_flag = False
ver_lst = tddfa.recon_vers(param_lst, roi_box_lst, dense_flag=dense_flag)
draw_landmarks(img, ver_lst, dense_flag=dense_flag)

x = ver_lst[0][0::6]
y = ver_lst[0][1::6]
# print(ver_lst[0][0::6]) # 横坐标
# print(ver_lst[0][1::6]) # 纵坐标
print(x[0][67])
print(y[0][67])
print(len(x[0]))
# print(ver_lst[0][2::6])
# ver_lst = [ver_lst]
# print(ver_lst[0][0::6])
b = np.array(ver_lst)
b.shape

In [ ]:
import numpy as np
# reconstruct vertices and visualizing dense landmarks
dense_flag = True
ver_lst = tddfa.recon_vers(param_lst, roi_box_lst, dense_flag=dense_flag)
draw_landmarks(img, ver_lst, dense_flag=dense_flag)

print(ver_lst)
print(ver_lst[0][0::6])
print(ver_lst[0][1::6])
# print(ver_lst[0][2::6])
a = np.array(ver_lst)
a.shape

In [ ]:
# reconstruct vertices and render
ver_lst = tddfa.recon_vers(param_lst, roi_box_lst, dense_flag=dense_flag)
render(img, ver_lst, tddfa.tri, alpha=0.6, show_flag=True);

In [ ]:
# reconstruct vertices and render depth
ver_lst = tddfa.recon_vers(param_lst, roi_box_lst, dense_flag=dense_flag)
depth(img, ver_lst, tddfa.tri, show_flag=True);

In [ ]:
# 1. running on still image, the options include: 2d_sparse, 2d_dense, 3d, depth, pncc, pose, uv_tex, ply, obj
# !python3 demo.py -f examples/inputs/emma.jpg --onnx # -o [2d_sparse, 2d_dense, 3d, depth, pncc, pose, uv_tex, ply, obj]
# !python3 demo.py -f /mnt/e/DataSet/screenshoot/face28.jpg --onnx -o 2d_dense
# !python3 demo.py -f /mnt/e/DataSet/xiaorongzhujianxiaoshi/face368_1.jpg --onnx -o uv_tex

# # 2. running on videos
# !python3 demo_video.py -f examples/inputs/videos/214.avi --onnx
# !python3 demo_video.py -f examples/inputs/videos/3DDFA-test2.mp4 --onnx -o 2d_sparse

# # 3. running on videos smoothly by looking ahead by `n_next` frames
# !python3 demo_video_smooth.py -f examples/inputs/videos/214.avi --onnx
# !python3 demo_video_smooth.py -f examples/inputs/videos/031taimei.mp4 --onnx -o 2d_sparse
# # 4. running on webcam
# !python3 demo_webcam_smooth.py --onnx